In [1]:
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.5 MB/s eta 0:00:00


In [2]:
import geemap
import ee

# Authenticate with Google Earth Engine
ee.Authenticate()  # Only needed once per session

# Initialize Earth Engine
# Replace 'gist-itproject2' with an Earth Engine project ID you have access to.
ee.Initialize(project='gist-itproject2')

In [3]:
import geemap
import ee

ee.Initialize(project='gist-itproject2')
Map = geemap.Map(center=[0, 160], zoom=3)

# Indo-Pacific countries
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
indo_pacific = countries.filter(ee.Filter.Or(
    ee.Filter.eq('wld_rgn', 'East Asia & Pacific'),
    ee.Filter.eq('wld_rgn', 'South Asia'),
    ee.Filter.eq('wld_rgn', 'Sub-Saharan Africa')
))

def add_country_popup(feature):
    name = feature.get('country_na')
    popup_html = ee.String('<b>Country:</b> ').cat(name).cat('<br><b>Government:</b> Democracy')
    return feature.set({'popup': popup_html})

indo_pacific_labeled = indo_pacific.map(add_country_popup)

Map.add_geojson(
    geemap.ee_to_geojson(indo_pacific_labeled),
    layer_name='Indo-Pacific Countries',
    info_mode='on_click',
    style={
        "color": "#003366",
        "fillColor": "#00FFFF",
        "weight": 2,
        "fillOpacity": 0.5
    }
)

# Country profile dictionary (used outside EE)
profiles = {
    "India": ("BJP-led coalition", "INDIA bloc", "Hinduism (79.8%), Islam (14.2%)"),
    "Australia": ("Labor Party", "Liberal-Nationals", "No Religion (38.7%), Catholic (19.6%)"),
    "Japan": ("LDP–Komeito", "CDP", "Shinto (48.6%), Buddhism (46.4%)"),
    "Indonesia": ("Gerindra", "PDI-P", "Muslim (87.2%), Protestant (7%)"),
    "Philippines": ("Bagong Pilipinas", "DuterTen", "Catholic (78.8%), Islam (6.4%)"),
    "Papua New Guinea": ("Pangu Party", "Independents", "Christianity (~96%)"),
    "Fiji": ("People’s Alliance–NFP–SODELPA", "FijiFirst", "Christianity (64.4%), Hinduism (27.9%)"),
}

# Define capital city features with popups built in Python
from shapely.geometry import Point
import geopandas as gpd

capitals = [
    {"lon": 77.2090, "lat": 28.6139, "name": "New Delhi", "country": "India", "pop": "32M"},
    {"lon": 139.6917, "lat": 35.6895, "name": "Tokyo", "country": "Japan", "pop": "37M"},
    {"lon": 151.2093, "lat": -33.8688, "name": "Canberra", "country": "Australia", "pop": "0.4M"},
    {"lon": 120.9842, "lat": 14.5995, "name": "Manila", "country": "Philippines", "pop": "13M"},
    {"lon": 106.8456, "lat": -6.2088, "name": "Jakarta", "country": "Indonesia", "pop": "34M"},
    {"lon": 147.1803, "lat": -9.4789, "name": "Port Moresby", "country": "Papua New Guinea", "pop": "0.4M"},
    {"lon": -179.194, "lat": -16.5782, "name": "Suva", "country": "Fiji", "pop": "0.09M"},
]

features = []
for c in capitals:
    ruling, opposition, religion = profiles.get(c["country"], ("Unknown", "Unknown", "Unknown"))
    popup = f"""
    <div style="font-family:Arial,sans-serif;padding:8px;border-radius:6px;
        background-color:#ffffff;min-width:250px;max-width:300px;
        border:1px solid #888; color:#000;">
        <h4 style="margin:0;color:#007BFF;font-weight:bold;">{c['name']}</h4>
        <p style="margin:6px 0 0 0;line-height:1.4;">
            <span style="font-weight:600;">Country:</span> {c['country']}<br>
            <span style="font-weight:600;">Population:</span> {c['pop']}<br>
            <span style="font-weight:600;">Latitude:</span> {c['lat']}<br>
            <span style="font-weight:600;">Longitude:</span> {c['lon']}<br>
            <span style="font-weight:600;">Ruling Party:</span> {ruling}<br>
            <span style="font-weight:600;">Opposition:</span> {opposition}<br>
            <span style="font-weight:600;">Top Religions:</span> {religion}
        </p>
    </div>
    """
    geom = ee.Geometry.Point([c["lon"], c["lat"]])
    feat = ee.Feature(geom, {"popup": popup})
    features.append(feat)

capital_fc = ee.FeatureCollection(features)

Map.add_geojson(
    geemap.ee_to_geojson(capital_fc),
    layer_name='Capital Popups',
    info_mode='on_click',
    style={
        "radius": 7,
        "color": "#ffffff",
        "fillColor": "#007BFF",
        "weight": 1.5,
        "fillOpacity": 1
    }
)

Map.set_control_visibility(True)
Map

Map(center=[0, 160], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…